In [1]:
import cv2
from keras.models import load_model
import numpy as np
import time
import random
from enum import IntEnum


class Action(IntEnum):
    Rock = 0
    Paper = 1
    Scissors = 2


def get_computer_selection():
    selection = random.randint(0, 2)
    return selection


def get_player_selection(frame):
    player_action = 0

    resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
    image_np = np.array(resized_frame)
    normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image
    data[0] = normalized_image
    prediction = model.predict(data)

    if prediction[0][0] > .7:
        player_action = 0

    elif prediction[0][1] > .7:
        player_action = 1

    elif prediction[0][2] > .7: 
        player_action = 2

    return player_action


def countdown(frame):
    t=3
    while t:
        cv2.putText(frame, "", (10, 50),  cv2.FONT_HERSHEY_COMPLEX, 1, (235, 52, 61), 1)
        if t == 3:
            print("ROCK")
            cv2.putText(frame, "ROCK", (10, 50),  cv2.FONT_HERSHEY_COMPLEX, 1, (235, 52, 61), 1)

        if t == 2:
            print("PAPER")
            cv2.putText(frame, "PAPER", (10, 50),  cv2.FONT_HERSHEY_COMPLEX, 1, (235, 52, 61), 1)
            
        if t == 1:
            print("SCISSORS")
            cv2.putText(frame, "SCISSORS", (10, 50),  cv2.FONT_HERSHEY_COMPLEX, 1, (235, 52, 61), 1)

        cv2.waitKey(1000)
        t -= 1
    print('SHOOT')
    cv2.putText(frame, "SHOOT", (10, 50),  cv2.FONT_HERSHEY_COMPLEX, 1, (235, 52, 61), 1)



def play_result(frame, player_action, computer_action):
    victories = {
        Action.Rock: [Action.Scissors],  # Rock beats scissors
        Action.Paper: [Action.Rock],  # Paper beats rock
        Action.Scissors: [Action.Paper]  # Scissors beats paper
    }

    defeats = victories[player_action]
    if player_action == computer_action:
        print(f"Both players selected {player_action.name}. It's a tie!")
        cv2.putText(frame, f"Both players selected {player_action.name}. It's a tie!", (10, 300),  cv2.FONT_HERSHEY_COMPLEX, 1, (235, 52, 61), 1)
        return 0
    elif computer_action in defeats:
        print(f"{player_action.name} beats {computer_action.name}! You win!")
        cv2.putText(frame, f"{player_action.name} beats {computer_action.name}! You win!", (10, 300),  cv2.FONT_HERSHEY_COMPLEX, 1, (235, 52, 61), 1)
        return 1
    else:
        print(f"{computer_action.name} beats {player_action.name}! You lose.")
        cv2.putText(frame, f"{player_action.name} beats {computer_action.name}! You lose!", (10, 300),  cv2.FONT_HERSHEY_COMPLEX, 1, (235, 52, 61), 1)
        return 2


def play_game():
    game_score = [0, 0]

    started = False

    while True:
        ret, frame = cap.read()
        cv2.putText(frame, "Press 'p' to play!", (10, 50),  cv2.FONT_HERSHEY_COMPLEX, 1, (235, 52, 61), 1)
        if not ret:
            continue
        
        #countdown(frame)

        #cv2.putText(frame, "TEST", (10, 50),  cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 0), 1)
        

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
#        countdown(frame)

        resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
        image_np = np.array(resized_frame)
        normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image
        data[0] = normalized_image
        prediction = model.predict(data)
        cv2.imshow('frame', frame)
        
        if started == False:
            if cv2.waitKey(1) & 0xFF == ord('p'):
                started = True
                print(started)

        if started:
            player_choice = 0
            
            if prediction[0][0] > .7:
                player_choice = 0

            elif prediction[0][1] > .7:
                player_choice = 1

            elif prediction[0][2] > .7: 
                player_choice = 2

            print(started, player_choice, prediction)
            started = False

            player_action = Action(player_choice)
            computer_action = Action(get_computer_selection())
            print(f"\nYou chose {player_action.name}, \
                computer chose {computer_action.name}.\n")

            result = play_result(frame, player_action, computer_action)

            if result == 1:
                i = 0
            elif result == 2:
                i = 1
            else:
                continue

            game_score[i] += 1

            if game_score[0] == 3 or game_score[1] == 3:
                if game_score[0] == 3:
                    print("YOU WIN THE MATCH", game_score)
                else:
                    print("CPU WIN THE MATCH", game_score)
                break

            print(game_score)

    cap.release()
    # Destroy all the windows
    cv2.destroyAllWindows()    


model = load_model('keras_model.h5')
cap = cv2.VideoCapture(0)
ret = cap.set(cv2.CAP_PROP_FRAME_WIDTH, 600.0) 
ret = cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 300.0)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

play_game()


2022-02-04 19:52:05.022264: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


True
True 1 [[3.7443527e-08 9.9999750e-01 2.5434135e-06 2.2482799e-10]]

You chose Paper,                 computer chose Rock.

Paper beats Rock! You win!
[1, 0]
True
True 1 [[1.0865950e-03 9.9886119e-01 3.7248188e-05 1.5031225e-05]]

You chose Paper,                 computer chose Rock.

Paper beats Rock! You win!
[2, 0]
True
True 1 [[9.0622474e-05 9.9892575e-01 8.1764272e-04 1.6601330e-04]]

You chose Paper,                 computer chose Paper.

Both players selected Paper. It's a tie!
True
True 0 [[0.7966433  0.18201089 0.00879829 0.0125475 ]]

You chose Rock,                 computer chose Rock.

Both players selected Rock. It's a tie!
True
True 1 [[1.7310566e-01 8.1440747e-01 1.2198515e-02 2.8833791e-04]]

You chose Paper,                 computer chose Scissors.

Scissors beats Paper! You lose.
[2, 1]
True
True 1 [[0.1478712  0.8026338  0.03387164 0.01562331]]

You chose Paper,                 computer chose Paper.

Both players selected Paper. It's a tie!
True
True 1 [[2.163633